# 10.6 • Artificial Intelligence Beyond Buzzwords (Hippo Edition)

When you hear **AI**, they often think of **ChatGPT** or **Grok**. 
But AI is a much bigger toolbox. Think of it like the different instruments in a lab:

- **Search algorithms** → finding a path through possibilities, like a hippo feeder robot choosing a route through the reserve.
- **Probabilistic reasoning** → handling uncertainty, like guessing if a hippo’s cough comes from diet or infection.
- **Decision-making under uncertainty** → planning actions when outcomes are not certain, like crossing a muddy shortcut.
- **Machine learning** → fitting models from data (see Section 10.5).

In this notebook we’ll explore small, tangible demos:

1. **Search**: BFS vs A* on a hippo feeding map.
2. **Decisions under uncertainty (MDP)**: should the robot feeder risk the muddy shortcut?
3. **Probabilistic reasoning (Bayesian net)**: inferring diet from symptoms.
4. **Learning & LLMs**: how this connects back to ChatGPT and modern AI.

In [ ]:
import numpy as np, heapq
import matplotlib.pyplot as plt
from collections import deque

np.random.seed(11088)

## 1) Problem-solving as search

Imagine a **hippo feeder robot** navigating a zoo map.

- Obstacles are walls and fences.
- The robot must find the shortest safe path to the hippos’ enclosure.

We compare two algorithms:

- **BFS**: explores in waves, ignoring how close it is to the goal.
- **A***: uses a heuristic (distance to goal) to guide exploration.

**Key idea**: If the heuristic never overestimates, A* finds an optimal path faster than BFS.

In [ ]:
def bfs(grid, start, goal):
    H, W = grid.shape
    Q = deque([start]); came = {start: None}; vis = {start}
    while Q:
        r, c = Q.popleft()
        if (r, c) == goal: break
        for dr, dc in [(1,0),(-1,0),(0,1),(0,-1)]:
            nr, nc = r+dr, c+dc
            if 0 <= nr < H and 0 <= nc < W and grid[nr, nc] == 0 and (nr, nc) not in vis:
                came[(nr, nc)] = (r, c)
                vis.add((nr, nc))
                Q.append((nr, nc))
    path = []
    cur = goal
    while cur is not None:
        path.append(cur)
        cur = came.get(cur)
    return path[::-1]

def astar(grid, start, goal):
    H, W = grid.shape
    def h(a, b): return abs(a[0]-b[0]) + abs(a[1]-b[1])  # Manhattan distance
    openq = [(h(start, goal), 0, start, None)]
    came = {}; g = {start: 0}; vis = set()
    while openq:
        f, cost, cur, par = heapq.heappop(openq)
        if cur in vis: continue
        vis.add(cur); came[cur] = par
        if cur == goal: break
        for d in [(1,0),(-1,0),(0,1),(0,-1)]:
            nr, nc = cur[0]+d[0], cur[1]+d[1]
            if 0 <= nr < H and 0 <= nc < W and grid[nr, nc] == 0:
                ng = cost + 1
                if ng < g.get((nr, nc), 1e9):
                    g[(nr, nc)] = ng
                    heapq.heappush(openq, (ng+h((nr,nc), goal), ng, (nr,nc), cur))
    path = []
    cur = goal
    while cur is not None:
        path.append(cur)
        cur = came.get(cur)
    return path[::-1]

# Create a small map
grid = np.zeros((20,20), int)
grid[5:15,10] = 1
grid[10,10] = 0  # a gate
start, goal = (0,0), (19,19)

p_bfs = bfs(grid, start, goal)
p_astar = astar(grid, start, goal)

plt.figure(figsize=(5,5))
plt.imshow(grid, cmap='gray_r')
rb, cb = zip(*p_bfs)
ra, ca = zip(*p_astar)
plt.plot(cb, rb, 'b--', label='BFS')
plt.plot(ca, ra, 'r-', label='A*')
plt.legend(); plt.title('Hippo feeder robot: BFS vs A*'); plt.show()

**Observation**  
BFS finds *a* path but wastes time exploring irrelevant areas.  
A* heads straight toward the goal, expanding fewer nodes.

👉 In AI, search underpins planning: routing delivery trucks, optimising lab sample flows, or navigating robots.

## 2) Decisions under uncertainty: Markov Decision Processes (MDPs)

Now our hippo feeder robot faces a choice:

- **Shortcut through the mud**: shorter, but 10% chance of slipping and losing progress.
- **Long paved path**: safe but longer.

This is a **Markov Decision Process (MDP)**: states, actions, transitions, rewards.

We use **value iteration** to compute the optimal policy.

In [ ]:
S = [(r,c) for r in range(3) for c in range(3)]
A = [(1,0), (-1,0), (0,1), (0,-1)]
goal = (2,2); gamma = 0.95

def step(s, a, slip=0.1):
    r,c = s; dr,dc = a
    main = (r+dr,c+dc); slip_alt = (r+dc,c+dr)
    nxt = []
    for i, (nr,nc) in enumerate([main, slip_alt]):
        if 0 <= nr < 3 and 0 <= nc < 3:
            p = 1-slip if i==0 else slip
            nxt.append(((nr,nc), p))
        else:
            p = 1-slip if i==0 else slip
            nxt.append(((r,c), p))
    return nxt

R = lambda s: 1.0 if s==goal else -0.04

V = {s:0.0 for s in S}
for _ in range(200):
    V = {s: (0 if s==goal else max(sum(p*(R(s2)+gamma*V[s2]) for s2,p in step(s,a)) for a in A)) for s in S}

Pi = {}
for s in S:
    if s == goal:
        Pi[s] = (0,0)
        continue
    Pi[s] = max(A, key=lambda a: sum(p*(R(s2)+gamma*V[s2]) for s2,p in step(s,a)))

np.array([[Pi[(r,c)] for c in range(3)] for r in range(3)])

**Interpretation**  
The arrows show the robot’s policy. When slip risk is low, it takes shortcuts.  
If slip risk increases, it may prefer safer routes.

👉 In health/nutrition: MDPs describe treatment planning, where each action has uncertain outcomes.

## 3) Probabilistic reasoning: a simple Bayesian network

Suppose we model:

- **High sugar diet (S)** can cause **hyperactivity (H)** and **dental issues (D)**.

If we observe a hippo is hyperactive, how does this update our belief about sugar intake and dental risk?

This is a **Bayesian network**: directed links between variables, with conditional probabilities.

In [ ]:
P_S = {1:0.3, 0:0.7}  # prior: 30% high sugar
P_H_given_S = {1:{1:0.6, 0:0.4}, 0:{1:0.2, 0:0.8}}
P_D_given_S = {1:{1:0.5, 0:0.5}, 0:{1:0.1, 0:0.9}}

# Observe H=1 (hyperactive)
unnorm = {s: P_H_given_S[s][1]*P_S[s] for s in [0,1]}
Z = sum(unnorm.values())
post_S = {s: unnorm[s]/Z for s in [0,1]}

# Predictive for D given H=1
P_D1 = sum(P_D_given_S[s][1]*post_S[s] for s in [0,1])

post_S, P_D1

**Result**  
Observing hyperactivity makes a high sugar diet more likely, and thus increases the inferred risk of dental issues.

👉 Bayesian reasoning is how AI systems combine noisy evidence into coherent probabilities (medical diagnosis, food safety monitoring, etc.).

## 4) Where does ChatGPT fit in?

- **Search** → Classical pathfinding and planning.
- **MDPs/RL** → Decisions under uncertainty, like treatment planning.
- **Bayesian nets** → Reasoning about hidden causes.
- **Machine learning (10.5)** → fitting predictive models from data.

**Large Language Models (LLMs)** like ChatGPT belong to the *learning* family. They learn statistical patterns in text.  
But for safety-critical systems (e.g. medical decision support, diet planning, or autonomous feeders), we often combine LLMs with **search, reasoning, and decision frameworks** to ensure reliability.

---
### Exercises
1. Change the A* heuristic to Euclidean distance. Is it still admissible? Compare the paths.
2. Increase slip in the MDP to 0.3. How does the policy change?
3. Extend the Bayes net with a variable `Exercise (E)` that reduces dental risk. Compute `P(D|H=1,E=1)`.

### Takeaways
- **Search** = goal-directed behaviour.
- **Probabilistic inference** = reasoning under uncertainty.
- **Decision-making** = optimising actions with trade-offs.
- **Learning** = using data to fit models.
- Modern AI (LLMs) = powerful learners, but classical AI tools remain essential for trustworthy systems.